# 2022-48 Census Building Permit ETL QC
Test Plan: https://sandag.sharepoint.com/:o:/r/qaqc/_layouts/15/Doc.aspx?sourcedoc=%7Bc2c3f693-42fd-4ef1-be35-d81805feb277%7D&action=editnew

## Library Imports

In [3]:
import os 
import pandas as pd
import numpy as np
import glob
import pyodbc
import gc

## Downloading Raw Data
R:\DPOE\Census\Census Building Permits\Source\raw\co2017a.txt 

R:\DPOE\Census\Census Building Permits\Source\raw\co2018a.txt 

R:\DPOE\Census\Census Building Permits\Source\raw\co2019a.txt 

R:\DPOE\Census\Census Building Permits\Source\raw\co2020a.txt 

R:\DPOE\Census\Census Building Permits\Source\raw\co2021a.txt 

In [4]:
# Downloading
data = pd.read_csv('R:/DPOE/Census/Census Building Permits/Source/raw/co2017a.txt')

In [5]:
data

,Survey,FIPS,FIPS.1,Region,Division,County,Unnamed: 6,1-unit,Unnamed: 8,Unnamed: 9,...,1-unit rep,Unnamed: 20,Unnamed: 21,2-units rep,Unnamed: 23,Unnamed: 24,3-4 units rep,Unnamed: 26,Unnamed: 27,5+units rep
Date,State,County,Code,Code,Name,Bldgs,Units,Value,Bldgs,Units,...,Value,Bldgs,Units,Value,Bldgs,Units,Value,Bldgs,Units,Value
2017,01,001,3,6,Autauga County,188,188,51711467,0,0,...,51711467,0,0,0,0,0,0,0,0,0
2017,01,003,3,6,Baldwin County,2207,2207,449821615,46,92,...,443998177,46,92,4240286,32,98,4034508,3,20,2091690
2017,01,005,3,6,Barbour County,3,3,901000,0,0,...,901000,0,0,0,0,0,0,0,0,0
2017,01,007,3,6,Bibb County,10,10,2551290,0,0,...,490000,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017,56,037,4,8,Sweetwater County,67,67,18053711,0,0,...,18053711,0,0,0,0,0,0,2,38,3732384
2017,56,039,4,8,Teton County,143,143,180463507,13,26,...,169595375,8,16,2400000,0,0,0,1,5,850000
2017,56,041,4,8,Uinta County,42,42,9324500,0,0,...,7564500,0,0,0,0,0,0,0,0,0
2017,56,043,4,8,Washakie County,4,4,884000,0,0,...,884000,0,0,0,0,0,0,0,0,0


## Downloading Staging Data

In [ ]:
def download_data(path, year):
    '''This function downloads both the source and staging data'''
    # Source Data (R Drive)
    files_list = glob.glob(path + f"/*{year}*")
    df_list = []
    for filename in files_list:
        df = pd.read_csv(filename, index_col=None, header=0)
        df_list.append(df)

    # Staging Data (SQL)
    conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=DDAMWSQL16.sandag.org;'
                      'Database=dpoe_stage;'
                      'Trusted_Connection=yes;')
    
    query ="SELECT *"\
        "FROM [dpoe_stage].[veh_reg_dmv].[fact]"\
        f"WHERE yr = {year};"

    return pd.concat(df_list, axis=0, ignore_index=True), pd.read_sql_query(query, conn)

## Downloading the Production Data